In [4]:
import pandas as pd
import requests

# pd.set_option("display.max_rows", 10000)

admin = pd.read_csv("../../data/행정구역코드.csv")
admin["행정구역코드"] = admin["행정구역코드"].astype(str)

# 수도권 + 광역시 제외
exclude_prefix = ["11", "31", "21", "22", "23", "24", "25", "26"]
sigun = admin[~admin["행정구역코드"].str[:2].isin(exclude_prefix)].copy()

objL1 = "+".join(sigun["행정구역코드"].tolist()) + "+"


In [5]:
BASE_URL = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

params = {
    "method": "getList",
    "apiKey": "ZWRiNzEyMzEwNGI5OWQ3NzcxNGM1MDNiOGJkOTQ0Y2M=",
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "itmId": "T001+",
    "objL1": objL1,
    "objL2": "ALL",
    "format": "json",
    "jsonVD": "Y",
    "prdSe": "Y",
    "newEstPrdCnt": "3"   # 최근 3개년
}

response = requests.get(BASE_URL, params=params)
df_raw = pd.DataFrame(response.json())

In [6]:
df = df_raw[[
    "PRD_DE",   # 연도
    "C1",       # 행정코드
    "C1_NM",    # 지역명
    "C2_NM",    # 지표명 (순이동률 / 순이동)
    "DT"        # 값
]].copy()

df.columns = ["연도", "행정코드", "지역명", "지표", "값"]

df["연도"] = df["연도"].astype(int)
df["행정코드"] = df["행정코드"].astype(str)
df["값"] = pd.to_numeric(df["값"], errors="coerce")

In [ ]:
# 시/군 코드 기준 병합
# df = df.merge(
#     admin[["행정구역코드", "시도명"]],
#     left_on="행정코드",
#     right_on="행정구역코드",
#     how="left"
# )

In [7]:
rate_df = df[df["지표"].str.contains("순이동률")].copy()
count_df = df[df["지표"].str.contains("순이동") & ~df["지표"].str.contains("률")].copy()

rate_df.rename(columns={"값": "청년순이동률"}, inplace=True)
count_df.rename(columns={"값": "청년순이동"}, inplace=True)

merged = rate_df.merge(
    count_df[["연도", "행정코드", "청년순이동"]],
    on=["연도", "행정코드"],
    how="inner"
)

summary = (
    merged
    .groupby(["행정코드", "지역명"])
    .agg(
        순이동_1년=("청년순이동", lambda x: x.iloc[-1]),
        순이동_2년=("청년순이동", lambda x: x.iloc[-2:].sum()),
        순이동_3년=("청년순이동", "sum"),
        평균_청년순이동률=("청년순이동률", "mean")
    )
    .reset_index()
)


In [ ]:
# 청년 유입 비율이 높은 지역 TOP 20
top_rate = (
    summary
    .sort_values("평균_청년순이동률", ascending=False)
    .head(20)
)

display(top_rate)

In [ ]:
top_count = (
    summary
    .sort_values("순이동_3년", ascending=False)
    .head(20)
)

display(top_count)

NameError: name 'result' is not defined